# LangChain: Evaluation

## Connect to OpenAI and define our model

Create an OpenAI API key for this project at [https://platform.openai.com/account/api-keys](https://platform.openai.com/account/api-keys). 

Copy `.env.sample` to `.env` and replace `<YOUR_OPENAI_API_KEY>` with the one you created.

Let's get started!

Let's make sure that we can read our OpenAI key from the `.env` file we created.

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Specify our model
llm_model="gpt-3.5-turbo-1106"

## Create our Q&A application

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch

In [ ]:
file = '../data/OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
data = loader.load()

In [ ]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [ ]:
llm = ChatOpenAI(temperature = 0.0, model=llm_model)
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

### Coming up with test datapoints

In [ ]:
data[10]

In [ ]:
data[11]

### Hard-coded examples

In [ ]:
examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

### LLM-generated examples

In [ ]:
# We can use QAGenerateChain to generate a chain that will look at our documents and generate answers to questions
from langchain.evaluation.qa import QAGenerateChain

In [ ]:
example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI(model=llm_model))

In [ ]:
# the warning below can be safely ignored

In [ ]:
# Use apply_and_parse to generate a dictionary of examples
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

In [ ]:
new_examples[0]

In [ ]:
new_examples

In [ ]:
data[0]

### Combine examples

In [ ]:
examples += new_examples

In [ ]:
# TODO: This code is not working with current versions of langchain
# qa.run(examples[0]["query"])

## Manual Evaluation

## LLM assisted evaluation

## LangChain evaluation platform